In [1]:
import numpy as np
import json
import re
import tensorflow as tf
import warnings
import random

import spacy

nlp = spacy.load("en_core_web_sm")

warnings.filterwarnings('ignore')

In [2]:
'''reading data'''
with open('intent.json', 'rb') as file:
    data = json.load(file)

In [3]:
'''preprocessing'''
def pre_processing(line):
    line = re.sub(r'[^a-zA-z.?!\']',' ',line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [4]:
'''get text and intent title from json data'''
inputs, targets = [],[]
cls = []
intent_doc = {}

for i in data['intents']:
    if i['intent'] not in cls:
        cls.append(i['intent'])

    if i['intent'] not in intent_doc:
        intent_doc[i['intent'] ] = []
    
    for text in i['text']:
        inputs.append(pre_processing(text))
        targets.append(i['intent'])

    for response in i['responses']:
        intent_doc[i['intent']].append(response)

In [5]:
print(inputs, targets)
print(cls)
print(intent_doc)

['Hi', 'Hi there', 'Hola', 'Hello there', 'Hey', 'My user is Bryan', 'This bryan', 'I am Bryan'] ['Greeting', 'Greeting', 'Greeting', 'Greeting', 'Greeting', 'GreetingResponse', 'GreetingResponse', 'GreetingResponse']
['Greeting', 'GreetingResponse']
{'Greeting': ['hi human, please tell me your geniSys user', 'Hello hu, please tell me your Greeting user'], 'GreetingResponse': ['Great! Hi <HUMAN>, how cant I help?', 'Cool Hi <HUMAN>']}


In [6]:
'''tokenize data'''
def token_data(inp_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    tokenizer.fit_on_texts(inp_list)

    inp_seq = tokenizer.texts_to_sequences(inp_list)

    '''adding padding'''
    inp_seq = tf.keras.preprocessing.sequence.pad_sequences(inp_seq, padding='pre')

    return tokenizer, inp_seq

'''preprocesss input data'''
tokenizer,inp_tensor = token_data(inputs)

In [7]:
print(tokenizer)
print(inp_tensor)

[[ 0  0  0  3]
 [ 0  0  3  4]
 [ 0  0  0  5]
 [ 0  0  6  4]
 [ 0  0  0  7]
 [ 8  9 10  2]
 [ 0  0 11  2]
 [ 0 12 13  2]]


In [8]:
def cr_cat_target(targets):
    word = {}
    cat_t = []
    counter = 0

    for trg in targets:
        if trg not in word:
            word[trg] = counter
            counter += 1
        cat_t.append(word[trg])

    cat_tensor = tf.keras.utils.to_categorical(cat_t, num_classes = len(word), dtype ="int32")
    return cat_tensor, dict((v,k) for k, v in word.items())

'''preprocess output data'''
target_tensor, target_idx_word= cr_cat_target(targets)

In [9]:
print(target_tensor)
print(target_idx_word)

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]
{0: 'Greeting', 1: 'GreetingResponse'}


In [10]:
f'input shape: {np.shape(inp_tensor)} and output shape: {target_tensor.shape}'

'input shape: (8, 4) and output shape: (8, 2)'

In [12]:
'''Build model'''
'''hyperparameters'''
epochs=50
vocab_size = len(tokenizer.word_index)+1
embed_dim = 512
units =128
target_len = target_tensor.shape[1]

'''Model'''
model = tf.keras.models.Sequential([
    ##Embedding layer
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    ##hidden layer
    tf.keras.layers.Dense(units,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    ##classification Layer
    tf.keras.layers.Dense(target_len, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 512)         7168      
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 696,706
Trainable params: 696,706
Non-trainable params: 0
________________________________________________

In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
print(inp_tensor)
print(target_tensor)

[[ 0  0  0  3]
 [ 0  0  3  4]
 [ 0  0  0  5]
 [ 0  0  6  4]
 [ 0  0  0  7]
 [ 8  9 10  2]
 [ 0  0 11  2]
 [ 0 12 13  2]]
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]


In [16]:
ealy_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


'''training'''
model.fit(inp_tensor, 
    target_tensor, 
    epochs=epochs,
    callbacks=ealy_stop)

Epoch 1/50
1/1 [==============================] - 0s 54ms/step - loss: 1.7061e-05 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 40ms/step - loss: 3.8147e-06 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 40ms/step - loss: 2.2352e-06 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 37ms/step - loss: 5.3346e-06 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 46ms/step - loss: 2.0713e-06 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 45ms/step - loss: 1.2666e-06 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 46ms/step - loss: 4.6342e-06 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 41ms/step - loss: 1.1176e-06 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 68ms/step - loss: 1.7285e-06 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 47ms/step - loss: 4.3213e-07 - accura

In [21]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))

    for token in doc :
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)

    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis = 1)

    rest = random.choice(intent_doc[target_idx_word[pred_class[0]]])
    return rest, target_idx_word[pred_class[0]]

In [43]:
response('i am andres')

('Cool Hi <HUMAN>', 'GreetingResponse')